In [1]:
import requests
import pandas as pd
import ivolatility as ivol
import matplotlib.pyplot as plt
import numpy as np
import holidays
import time

# ivolatility connection
apiKey = '7ANa7nqtg2RWt7n5'
#apiKey = 'fFF8EEL54op7k0DN'
ivol.setLoginParams(apiKey=apiKey)

In [2]:
# Variables
symbol = 'DELL'
date = '2024-04-30'
acciones = ["TSLA", "BA", "FDX", "BAC", "DIS"]

Datos complementarios 

In [3]:
# Get stock EOD Price
def fetch_eod_stock_prices(api_key, symbol, date):
    url = f"https://restapi.ivolatility.com/equities/eod/stock-prices?apiKey={api_key}&symbol={symbol}&date={date}"
    response = requests.get(url)
    data = response.json()
    # Assuming the relevant data is under the 'data' key
    df = pd.DataFrame(data['data'])
    return df
Current = fetch_eod_stock_prices('fFF8EEL54op7k0DN', symbol, date)
Current

,record_no,date,symbol,exchange,open,high,low,close,volume,unadjusted_close
0,1,2024-04-30,DELL,NYSE,126.25,126.99,123.84,124.64,4420082,124.64


In [4]:
getMarketData = ivol.setMethod('/equities/eod/hv')
marketData = getMarketData(symbol=symbol, date=date)
df_H = pd.DataFrame(marketData)
df_H

,record_no,symbol,exchange,date,price,10d HV,20d HV,30d HV,60d HV,90d HV,120d HV,150d HV,180d HV
0,1,DELL,NYSE,2024-04-30,124.64,0.391822,0.517707,0.469334,0.713422,0.613079,0.548371,0.503624,0.520878


Funciones Get IV and Get HV

In [5]:
def incremental_pause(initial_pause=10, increment=3, state={'current_pause': None}):
    """
    Pause the execution incrementally each time the function is called.

    Args:
    - initial_pause (int): Initial pause duration in seconds.
    - increment (int): Amount to increment the pause by with each call.
    - state (dict): Mutable default argument to maintain state across function calls.
    """
    if state['current_pause'] is None:
        state['current_pause'] = initial_pause
    else:
        state['current_pause'] += increment

    print(f"Pausing for {state['current_pause']} seconds...")
    time.sleep(state['current_pause'])

In [6]:
def create_working_days(start_date, end_date, country='US'):
    # Generate a date range for all days between start and end
    all_days = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Fetch holidays for the given country in the range of years
    us_holidays = holidays.CountryHoliday(country, years=[start_date.year, end_date.year])
    
    # Filter out weekends and holidays, and format dates as strings
    working_days = [day.strftime('%Y-%m-%d') for day in all_days if day.weekday() < 5 and day not in us_holidays]
    
    return working_days

# Example usage: create a list of working days from January 1, 2024, to May 3, 2024
date_range = create_working_days(pd.Timestamp('2023-01-01'), pd.Timestamp('2024-05-03'))

# List of additional federal holidays to consider
federal_holidays_list = [
    '2024-01-01', '2024-01-15', '2024-02-19','2024-03-29', '2024-05-27', '2024-07-04',
    '2024-09-02', '2024-10-14', '2024-11-11', '2024-11-28', '2024-12-25'
]

# Convert both the generated date range and federal holidays list to sets
date_range_set = set(date_range)
federal_holidays_set = set(federal_holidays_list)

# Subtract the federal holidays from the date range
working_days_final = date_range_set - federal_holidays_set

# Convert the set back to sorted list
date_range = sorted(list(working_days_final))

print("Working days excluding all holidays:")
print(len(date_range))

Working days excluding all holidays:
335


In [7]:
# Get Nearest Option Price
getMarketData = ivol.setMethod('/equities/eod/nearest-option-tickers')
marketData = getMarketData(symbol=symbol, startingDate=date, dte=60, moneyness=0, callPut='P')
df_near = pd.DataFrame(marketData)
first_option_id = df_near.iloc[0]["option_id"]
first_option_symbol= df_near.iloc[0]["option_symbol"]
print(first_option_symbol)
df_near

DELL  240621P00120000


,option_id,option_symbol,comment
0,130694032,DELL 240621P00120000,NaN
1,130750066,DELL 240621P00125000,NaN


In [8]:
def get_nearest_option_price(symbol, date):
    """
    Fetch the nearest option price for a given symbol and date.

    Parameters:
    symbol (str): Stock symbol.
    date (str): Starting date in YYYY-MM-DD format.

    Returns:
    tuple: A tuple containing the first option symbol and its DataFrame.
    """
    # Set method and fetch market data
    getMarketData = ivol.setMethod('/equities/eod/nearest-option-tickers')
    marketData = getMarketData(symbol=symbol, startingDate=date, dte=60, moneyness=0, callPut='P')
    
    # Create DataFrame from market data
    df_near = pd.DataFrame(marketData)
    
    # Extract the first option's ID and symbol
    first_option_id = df_near.iloc[0]["option_id"]
    first_option_symbol = df_near.iloc[0]["option_symbol"]
    
    # Print the first option symbol (optional)
    print(first_option_symbol)
    
    return first_option_symbol

In [9]:
first_option_symbol = get_nearest_option_price(symbol, date)
#print(first_option_symbol)

DELL  240621P00120000


In [10]:
def getMarketData(symbol, date):

    getMarketData = ivol.setMethod('/equities/eod/hv')
    marketData = getMarketData(symbol=symbol, date=date)
    df_H = pd.DataFrame(marketData)
    # Asignar el valor de '180d HV' a la variable HistVol
    HistVol = df_H['180d HV'].iloc[0] 
    return HistVol

In [11]:
def get_IV(first_option_symbol, date):
    """
    Obtiene la volatilidad implícita para una opción específica en una fecha dada, filtrando por columnas específicas.
    Retorna:
    Un DataFrame de pandas con la última entrada de la volatilidad implícita, incluyendo solo las columnas especificadas.
    """
    # Suponiendo que 'ivol' es un objeto global o está definido previamente
    # Configurar el método de obtención de datos del mercado
    getMarketData = ivol.setMethod('/equities/intraday/single-equity-optionsymbol-rawiv')
    marketData = getMarketData(optionSymbol=first_option_symbol, date=date, minuteType='HOUR')
    # Convertir los datos del mercado en un DataFrame
    df_IV = pd.DataFrame(marketData)
    # Filtrar las columnas deseadas
    columns = ['timestamp', 'stockSymbol', 'underlyingPrice', 'optionBidPrice', 'optionAskPrice','optionIv', 'optionDelta', 'optionGamma', 'optionTheta', 'optionVega']
    df_IV = df_IV[columns]
    
    return df_IV.tail(1)

Ejecución de la petición al servidor

In [ ]:
#date_range = ['2024-04-04']
# Iterar sobre el rango de fechas y obtener la volatilidad implícita para cada fecha
results = []
for date in date_range:
    symbol = symbol
    first_option_symbol = get_nearest_option_price(symbol, date)
    HistVol = getMarketData(symbol, date)
    iv_data = get_IV(first_option_symbol=first_option_symbol, date=date) 
    #HistVol = getMarketData(first_option_symbol=first_option_symbol, date=date)
    print(date,HistVol,iv_data,'Volat_hist',HistVol, first_option_symbol)
    print('\n')
    
    incremental_pause()
    #time.sleep(15)

    if not iv_data.empty:
        results.append(iv_data)
    else:
        print(f"No market data for {date.strftime('%Y-%m-%d')}")

# Concatenar todos los DataFrames en uno solo si hay resultados
if results:
    df = pd.concat(results, ignore_index=True)
    df['Opt_ticker'] = first_option_symbol
    df['Opt_HV'] = HistVol  
    df['Opt_HV'] = round(df['Opt_HV'], 3)
    df['IV_HV_Ratio'] = (df['optionIv'] / df['Opt_HV']) * 100
    df = df.round(2)
        
else:
    df = pd.DataFrame()  # Crea un DataFrame vacío si no hay datos de mercado
df

DELL  230317P00040000
2023-01-03 0.443761              timestamp stockSymbol  underlyingPrice  optionBidPrice  \
6  2023-01-03 16:00:00        DELL           40.915            2.23   

   optionAskPrice  optionIv  optionDelta  optionGamma  optionTheta  optionVega  
6            2.31    0.3768     -0.41708     0.058501     -0.01633     0.07088   Volat_hist 0.443761 DELL  230317P00040000


Pausing for 10 seconds...
DELL  230317P00040000
2023-01-04 0.44356              timestamp stockSymbol  underlyingPrice  optionBidPrice  \
6  2023-01-04 16:00:00        DELL           40.895            2.18   

   optionAskPrice  optionIv  optionDelta  optionGamma  optionTheta  optionVega  
6            2.24    0.3694     -0.41801     0.060164    -0.016097     0.07038   Volat_hist 0.44356 DELL  230317P00040000


Pausing for 13 seconds...
DELL  230317P00040000
2023-01-05 0.443318              timestamp stockSymbol  underlyingPrice  optionBidPrice  \
6  2023-01-05 16:00:00        DELL           40.865    

In [ ]:
df.to_csv(f'output/iteracion_{symbol}.csv', index=False)

In [ ]:
df.tail()

Graficación de Resultados

In [ ]:
# Assuming df is already defined with correct data and a datetime index.
symbol = df.iloc[0]['stockSymbol']

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(10, 12), gridspec_kw={'height_ratios': [2, 2, 1]})

# Plotting IV/HV Ratio
ax1.plot(df.index, df['IV_HV_Ratio'], color='blue', label='IV/HV Ratio')
ax1.set_ylabel('IV/HV Ratio')
ax1.legend(loc='upper left')
ax1.set_title(f'IV/HV Ratio and Underlying Price Analysis for {symbol}')  # Title for ax1

# Adding the twin axis for the underlying price
ax1_right = ax1.twinx()
ax1_right.plot(df.index, df['underlyingPrice'], color='green', label=f'{symbol} Underlying Price', alpha=0.4)
ax1_right.set_ylabel(f'Price of {symbol}')
ax1_right.legend(loc='upper right')

# Calculate and Highlight the 90th percentile of IV/HV Ratio
percentile_90_ivhv = df['IV_HV_Ratio'].quantile(0.9)
ax1.axhline(y=percentile_90_ivhv, color='blue', linestyle='--')
ax1.fill_between(df.index, df['IV_HV_Ratio'], percentile_90_ivhv, where=(df['IV_HV_Ratio'] >= percentile_90_ivhv), color='blue', alpha=0.3)
ax1.legend(loc='upper left', bbox_to_anchor=(0, 1.2))  # Adjust legend to fit

# Plotting Option IV on the third axis
ax3.plot(df.index, df['optionIv'], color='violet', alpha=0.5, label='Option IV')
ax3.set_ylabel('Option IV')
ax3.legend(loc='upper left')

# Plotting Option Ask Price on the second axis, highlight 90th percentile and add fill
percentile_90_ask = df['optionAskPrice'].quantile(0.9)
ax2.plot(df.index, df['optionAskPrice'], color='red', label='Option Ask Price', alpha=0.7)
ax2.axhline(y=percentile_90_ask, color='red', linestyle='--', label=f'90th Percentile Ask Price = {percentile_90_ask:.2f}')
ax2.fill_between(df.index, df['optionAskPrice'], percentile_90_ask, where=(df['optionAskPrice'] >= percentile_90_ask), color='red', alpha=0.5)
ax2.set_ylabel('Option Ask Price')
ax2.set_xlabel('Date')  # Ensure xlabel reflects the content of the index
ax2.legend(loc='upper left')

# Adjust layout to fit the subplots neatly within the figure window
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust the layout to provide space for the title

plt.show()

In [ ]:
# Assuming df is already loaded with a datetime index set
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(10, 12), gridspec_kw={'height_ratios': [3, 1, 1]})

# Plotting IV_HV_Ratio
ax1.plot(df.index, df['IV_HV_Ratio'], color='blue', label='IV/HV Ratio')
ax1.set_ylabel('IV/HV Ratio')
ax1.legend(loc='upper left')
ax1.set_title(f'IV/HV Ratio and Underlying Price Analysis for {symbol}')  # Title for ax1
ax1_right = ax1.twinx()
ax1_right.plot(df.index, df['underlyingPrice'], color='green', label=f'subyaciente {symbol}', alpha = 0.3)
ax1_right.set_ylabel(f'valor {symbol}')
ax1_right.legend(loc='upper right')
ax1.axhline(y=120, color='blue', linestyle='--', label='Mean IV/HV Ratio') 


# Plotting optionIv
ax2.plot(df.index, df['optionIv'], color='violet', alpha=0.5, label='Option IV')
ax2.set_ylabel('Option IV')
ax2.legend(loc='upper left')

# Plotting optionAskPrice
ax3.plot(df.index, df['optionAskPrice'], label='Option Ask Price')
ax3.set_ylabel('Option Ask Price')
ax3.set_xlabel('Date')  # Ensure xlabel reflects the content of the index
ax3.legend(loc='upper left')

# Adjust layout to fit the subplots neatly within the figure window
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust the layout to provide space for the title

plt.show()